In [5]:
import copy
from datetime import datetime

from pytket import Circuit, OpType
from sympy import Symbol

from qnexus.client import auth, circuits, projects, jobs

In [ ]:
auth.login()

## Create a Project with Properties to organize our workflow 

### Projects are a container, we'll use it here also to set our properties

In [ ]:
project_ref = projects.submit(
    name=f"VQE_example_{str(datetime.now())}", 
    description="A VQE done with qexus", 
    properties={}
)

### Properties can be used to store parameters for our VQE

In [3]:

projects.add_property(
    project_ref, 
    name="is_symbolic_circuit", 
    property_type="bool", 
    description=f"A label for symbolic circuits", 
    required=False
)

vqe_params = ["alpha", "beta", "gamma"]

for param_name in vqe_params:
    projects.add_property(
        project_ref, 
        name=param_name, 
        property_type="float", 
        description=f"Our VQE {param_name} parameter", 
        required=False
    )

## Prepare the code for our VQE loop

In [ ]:
alpha = Symbol("alpha")
beta = Symbol("beta")
gamma = Symbol("gamma")

symbolic_circ = Circuit(2,2).Rx(alpha, 0).Rx(-2*beta, 1).CX(0, 1).add_gate(OpType.YYPhase, gamma, [0, 1]).measure_all()


symbolic_circuit_ref = circuits.submit(
    circuit=symbolic_circ, 
    project=project_ref,
    name="Original symbolic circuit",
    description="",
    properties={
        "alpha": True,
        "beta": True,
        "gamma": True,
        "is_symbolic_circuit": True,
    }
)

In [ ]:

current_s_map = {alpha: 0.3, beta:1.25, gamma:0.2}

def vqe_loop():
    
    iteration_count = 0

    vqe_result: dict = {}

    while iteration_count > 3:

        iteration_count +=1
        
        # get circuit and update params
        active_circ = copy.deepcopy(symbolic_circ)
        active_circ.symbol_substitution(current_s_map)

        active_circ_ref = circuits.submit(
            circuit=active_circ, 
            project=project_ref,
            name=f"VQE Circuit",
            description="A new circuit",
            properties={
                "alpha": current_s_map[alpha],
                "beta": current_s_map[beta],
                "gamma": current_s_map[gamma],
            }
        )


        # compile the circuit
        compile_job_ref = jobs.submit_compile_job(
            name=f"VQE_compile_job",
            circuits=active_circ_ref,
            optimisation_level=1,
            project=project_ref
        )

        # wait for the job TODO
        compiled_circuit_refs = jobs.compilation_results(compile_job_ref)

        active_execute_ref =jobs.submit_execute_job(
            name="My Execute Job",
            circuits=compiled_circuit_refs,
            project=project_ref,
            n_shots=[10]
        )

        # wait for the job TODO

        my_results = jobs.execution_results(active_execute_ref)
        result = my_results[0].get_result()

        vqe_result["result"] = result

        # use the result to exit or generate new params

        current_s_map[alpha] +=0.1
        current_s_map[beta] -=0.1
        current_s_map[gamma] +=0.3
    
    return vqe_result




### If our VQE loop terminates during execution (e.g. laptop runs out of battery) we can resume

# TODO